# Parser tests

In [1]:
[global]
from dsc.dsc_parser import DSC_Script as ds
from dsc.utils import FormatError

def test_format(obj, *args):
    try:
        return obj(*args)
    except Exception as e:
        if type(e).__name__ == 'FormatError':
            print(f'* {e}')
        else:
            raise
        return None

text0 = '''
DSC:
    run: simulate
'''

## Basic syntax

In [2]:
%run
[1]
# F: more exec output
text = text0 + '''
simulate: sim.R, t.R
    x: 1
    y: 2
    $out: x
'''
test_format(ds, text)

* Block ``simulate`` specifies ``1`` modules, yet ``2`` executables are provided. Please ensure they match.


In [3]:
%run
[2]
# F: grouped parameters is not allowed
text = text0 + '''
simulate: sim.R
    group_1:
        x: 1
        y: 2
    $out: x
'''
test_format(ds, text)

* Invalid decoration ``group_1``. Decorations must start with ``@`` symbol.


In [4]:
%run
[3]
# P: exec decoration
text = text0 + '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
    $out: x
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].dump()['input'].items()) == [('x', [1]), ('y', [2])]

In [5]:
%run
[4]
# F: grouped parameters is not allowed in exec decoration
text = text0 + '''
simulate: sim.R
    @simulate:
        group_1:
            x: 1
            y: 2
    $x: x
'''
res = test_format(ds, text)

* Invalid decoration ``group_1``. Decorations must start with ``@`` symbol.


In [6]:
%run
[5]
# F: alias is not a list
text = text0 + '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
    @ALIAS:
        x_1: x
    $out: x
'''
res = test_format(ds, text)

* Cannot find module ``simulate`` in @ALIAS specification ``['x_1']``.


In [7]:
%run
[vi_1]
# P: alias for specific module, both syntax supported
text = text0 + '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
    @ALIAS: 
        simulate: x_1 = x
    $out: x
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].dump()['input'].items()) == [('x', [1]), ('y', [2])]
assert list(res.modules['simulate'].plugin.alias_map.items()) == [('x', 'x_1')]

[vi_2]
text = text0 + '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
        @ALIAS: x_1 = x
    $out: x
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].dump()['input'].items()) == [('x', [1]), ('y', [2])]
assert list(res.modules['simulate'].plugin.alias_map.items()) == [('x', 'x_1')]

[vi_3]
text = text0 + '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
        @ALIAS: 
            *: x_1 = x
    $out: x
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].dump()['input'].items()) == [('x', [1]), ('y', [2])]
assert list(res.modules['simulate'].plugin.alias_map.items()) == [('x', 'x_1')]

In [8]:
%run
[7]
# F: invalid decoration / module
text = text0 + '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
    @test:
        x: 3
    $out: x
'''
res = test_format(ds, text)

* Undefined decoration ``@test/^test``.


In [9]:
%run
[8]
# F: invalid decoration / module
text = text0 + '''
simulate: sim.R
    @simulate, test:
        x: 1
        y: 2
    $out: x
'''
res = test_format(ds, text)

* Undefined decoration ``@test/^test``.


In [10]:
%run
[9]
# P: use global variable
text = '''
simulate: sim.R
    x: 1
    y: $(x)
    $out: x
DSC:
    run: simulate
    global:
        x: 2
'''
res = test_format(ds, text)
list(res.modules['simulate'].dump()['input'].items()) == [('x', [2]), ('y', [2])]
text = '''
simulate: sim.R
    x: 1
    y: $(x)[3]
    $out: x
DSC:
    run: simulate
    global:
        x: 1,2,3,4
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].dump()['input'].items()) == [('x', [1,2,3,4]), ('y', [3])]

In [11]:
%run
[10]
# P: alias partial list / dict
text = text0 + '''
simulate: sim.R
  x: 1
  y: 2
  @ALIAS: args = List(), arg1 = List(xvar = x, yy = y)
  $out: x
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].plugin.dump()['container_variables'].items()) == [('x', [None, 'xvar']), ('y', [None, 'yy'])]

In [12]:
%run
[11]
# P: filter decorator
text = text0 + '''
simulate: sim.R
    x: R{1:5}
    @FILTER:
        *: x < 3
    $out: x
'''
res = test_format(ds, text)

## Bad variable / module names

In [13]:
%run
[12]
# F: various bad names
text = '''
@@simulate: sim.R
    x: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    $x: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    .x: 1
DSC: 
    run: simulate
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    _x: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    x$: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    x.y: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    x_y: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    x.1: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    1: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    _: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    **: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    .: 1
'''
res = test_format(ds, text)

* Invalid module name ``?@simulate``
* Cannot find required section ``DSC``!
* Dot is not allowed for module / variable names, in ``.x``. Note that dotted names is not acceptable to Python and SQL. If this limitation is irrelevant to your problem, and you really cannot rename variable in your code, then at your own risk you can rename ``.x`` to, eg, ``name`` in DSC and use ``@ALIAS: .x = name``.
* Names cannot start or end with underscore, in ``_x``. Note that such naming convention is not acceptable to R. If this limitation is irrelevant to your problem, and you really cannot rename variable in your code, then at your own risk you can rename ``_x`` to, eg, ``name`` in DSC and use ``@ALIAS: _x = name``.
* ``$`` is not allowed in module / variable names, in ``x$``.
* Dot is not allowed for module / variable names, in ``x.y``. Note that dotted names is not acceptable to Python and SQL. If this limitation is irrelevant to your problem, and you really cannot rename variable in your code, t

## Module derivation

In [14]:
%run
[13]
# F: missing exec
text = text0 + '''
simulate: 
    x: R{1:5}
    $out: x
'''
res = test_format(ds, text)

* Invalid syntax ``simulate:``. Should be in the format of ``module names: module executables``


In [15]:
%run
[xiii_1]
# P: missing exec in derived
text = text0 + '''
base: sim.R
    x: 2
    $out: x
simulate(base): 
    x: R(1:5)
'''
res = test_format(ds, text)
assert res.modules['simulate'].dump()['command'] == 'sim.R'

[xiii_2]
text = text0 + '''
base: sim.R
    x: 2
    $out: x
simulate(base): t.R
    x: R(1:5)
'''
res = test_format(ds, text)
assert res.modules['simulate'].dump()['command'] == 't.R'

In [16]:
%run
[14]
# F: derive from compact
text = '''
normal, t: rnorm.R, rt.R
    n: 1000
    @normal:
        y: 5
        n: 6
    $x: x
    
simulate(normal, t): 
    mu: 1
DSC:
    run: test
'''
res = test_format(ds, text)

* Invalid base module name ``normal,t``. Base module has to be single module.


In [17]:
%run
[15]
# P: derive from one of compact
text = text0 + '''
normal, t: sim.R, t.R
    n: 1000
    @normal:
        y: 5
        n: 6
    $x: x
    
simulate(normal):
    mu: 1
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].dump()['input'].items()) == [('n', [6]), ('y', [5]), ('mu', [1])]

In [18]:
%run
[16]
# F: looped
text = text0 + '''
normal, t (shifted_normal): sim.R, t.R
    n: 1000
    @normal:
        y: 5
        n: 6
    $x: x
    
shifted_normal(normal):
    mu: 1
'''
res = test_format(ds, text)

* Looped block inheritance: normal(shifted_normal) and shifted_normal(normal)!


In [19]:
%run
[17]
# F: non-existing base
text = text0 + '''
base: sim.R
    x: 2
    $out: x
simulate(base1): 
    x: R(1:5)
'''
res = test_format(ds, text)

* Base block ``base1`` does not exist for simulate(base1)!


## Grouped parameters

In [20]:
%run
[18]
# P: grouped parameters
text = text0 + '''
simulate: sim.R
    (n,p): (1,2), (5,6)
    a,b: (3,4)
    t: 5
    $x: x
'''
res = test_format(ds, text)

## `R()` operator

In [21]:
%run
[19]
# P: R operator
text1 = text0 + '''
simulate: sim.R
    (n,p): (1,2), (5,6)
    $x: x
'''
res1 = test_format(ds, text1)
text2 = text0 + '''
simulate: sim.R
    (n,p): R{list(c(1,2), c(5,6))}
    $x: x
'''
res2 = test_format(ds, text1)
assert str(res1) == str(res2)

## CONF test

In [22]:
%run
[20]
text1 = text0 + '''
simulate: sim.R
    $x: x
    @CONF: work_dir = /tmp, exec_path = /tmp
'''
res = test_format(ds, text1)
assert res.modules['simulate'].path == ['/tmp']

In [23]:
%run
[25]
text1 = text0 + '''
simulate: sim.R
    $x: x
    @CONF: work_dir = /tmp, exec_path = (/tmp, ~/tmp), lib_path = ./, R_libs = (ashr@stephenslab/ashr (2.2.7+), psych)
'''
res = test_format(ds, text1)
assert res.modules['simulate'].path == ['/tmp', '~/tmp']
assert res.modules['simulate'].exe['header'] == 'library(ashr)\nlibrary(psych)\n'

## Input / output variable test

In [24]:
%run
[21]
# F: multiple input/output
text1 = text0 + '''
simulate: sim.R
    $x: x, 7
'''
res = test_format(ds, text1)
text1 = text0 + '''
simulate: sim.R
    x: $x, y
    $x: x
'''
res = test_format(ds, text1)

* Module output ``x`` cannot contain multiple elements ``['x', 7]``
* Module input ``x`` cannot contain multiple elements ``['$x', 'y']``


## Duplicates

In [25]:
%run
[22]
# F: various duplicates
text1 = text0 + '''
simulate: sim.R
    $x: x

simulate, simulate: t.R
    $x: 7
'''
res = test_format(ds, text1)
text1 = text0 + '''
simulate: sim.R
    $x: x

simulate: t.R
    $x: 7
'''
res = test_format(ds, text1)
text1 = text0 + '''
simulate: sim.R
    $x: x

t,t: t.R
    $x: 7
'''
res = test_format(ds, text1)
text1 = text0 + '''
simulate: sim.R
    $x: x

t,simulate: t.R
    $x: 7
'''
res = test_format(ds, text1)
text1 = text0 + '''
simulate: sim.R
    $x: x
    $x: y

t,simulate: t.R
    $x: 7
'''
res = test_format(ds, text1)

* Duplicate module in block ``simulate,simulate``.
* Duplicate block name ``simulate``.
* Duplicate module in block ``t,t``.
* Duplicate module definition ``simulate``


  in "<unicode string>", line 2, column 5:
        $x: x
        ^
found duplicate key ($x)
  in "<unicode string>", line 3, column 5:
        $x: y
        ^


* ``DSCStringIO.dsc`` has caused ``ConstructorError`` (see above).
Please ensure there is no duplicated variable names in modules. If this is related to string with quotes please remove them, or use the 'raw()' syntax.


## FILTER test

In [26]:
%run
[23]
# F: bad condition
text = text0 + '''
simulate: sim.R
    n: $y
    $x: x
    @FILTER: n < 3
'''
res = test_format(ds, text)
text = text0 + '''
simulate: sim.R
    n: 2
    p: 5
    $x: x
    @FILTER: n < 3
    @ALIAS: m = n
'''
res = test_format(ds, text)

* No parameter combination satisfies @FILTER ``n < 3``!


In [27]:
%run
[24]
# P: filter tests
text = text0 + '''
simulate, t: sim.R, t.R
    n: 100, 200, 300, 400, 500
    k: 0, 1
    @FILTER:
        simulate: (n <= 300 and k = 0) or (n > 300 and k = 1)
        t: n = 500
    $x: x
'''
res = test_format(ds, text)
assert (res.modules['simulate'].dump()['input_filter']) == '(_n <= 300 and _k == 0) or (_n > 300 and _k == 1)'
text = text0 + '''
simulate, t: sim.R, t.R
    n: 100, 200, 300, 400, 500
    k: 0, 1
    @FILTER: (n in [100,200,300] and k = 0)
    $x: x
'''
res = test_format(ds, text)
assert (res.modules['simulate'].dump()['input_filter']) == '(_n in (100,200,300) and _k == 0)'
text = text0 + '''
simulate, t: sim.R, t.R
    n: 100, 200, 300, 400, 500
    k: 0, 1
    @FILTER: 
        *: n in [100,200,300]
        t: n = 300
    $x: x
'''
res = test_format(ds, text)
assert (res.modules['simulate'].dump()['input_filter']) == '(_n in (100,200,300))'

In [28]:
%run
[25]
# F: parameter name conflict with output
text = text0 + '''
simulate: sim.R
    n: 100
    $n: n
'''
res = test_format(ds, text)
# F: parameter name conflict with output
text = text0 + '''
simulate: sim.R
    n: $n
    $n: n
'''
res = test_format(ds, text)

* Name ``n`` cannot be used for both parameter and output for module ``simulate``


In [29]:
%run
[26]
# F: parameter name conflict with output
text = text0 + '''
simulate: sim.R
    n: 100
    a: 5
    @ALIAS: m = n, m = a
    $out: n
'''
res = test_format(ds, text)

* Duplicated @ALIAS ``['m']`` in module ``simulate``
